<a href="https://colab.research.google.com/github/jsxhhyf/Optiver/blob/main/prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 环境配置

In [ ]:
!mkdir -p ~/.kaggle
!cp ./drive/MyDrive/Colab\ Notebooks/Kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!unzip ./drive/MyDrive/Colab\ Notebooks/Kaggle/optiver-realized-volatility-prediction/optiver-realized-volatility-prediction.zip

In [ ]:
!mkdir -p ./optiver-realized-volatility-prediction/parquet
# !cp ./drive/MyDrive/Colab\ Notebooks/Kaggle/preprocessed/* ./optiver-realized-volatility-prediction/parquet

In [ ]:
!pip install wandb
!pip install pytorch-lightning
!pip install tensor-sensor[torch]

## 导入数据

In [ ]:
# IMPORTS
import os, sys, random, datetime
import pandas as pd
# import modin.pandas as pd
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import *

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
import torchmetrics

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

from tqdm import tqdm, trange

import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# GLOBAL VARIABLES
BASE_DIRECTORY = "./optiver-realized-volatility-prediction"

In [ ]:
def expand_to_600(df: "dataframe") -> "dataframe":
    """
    :param: dataframe to be expanded
    :return: expanded dataframe
    """
    added_list = []
    cur = -1
    seconds_list = df.seconds_in_bucket.values
    nparray = df.values

    for i in range(600):
        if i in seconds_list:
            cur += 1
        else:
            temp = nparray[cur, :].copy()
            temp[1] = i
            temp = temp[np.newaxis, :]
            added_list.append(temp)
            
    if added_list:
        added_np = np.concatenate(added_list, axis=0)
        df_new = pd.DataFrame(added_np, columns=df.columns)
        df = pd.concat([df, df_new]).astype(np.float32)

    return df

In [ ]:
# book_train.parquet
SUB_DIRECTORY = "./book_train.parquet"

for id in tqdm(range(127)):
    book_train_list = []
    cur_path = os.path.join(SUB_DIRECTORY, f"stock_id={id}")
    if os.path.exists(cur_path):
        file = os.listdir(cur_path)[0]
        temp_df = pd.read_parquet(os.path.join(cur_path, file))
        temp_df["stock_id"] = id
        temp_df["WAP"] = (
            temp_df.bid_price1 * temp_df.ask_size1
            + temp_df.ask_price1 * temp_df.bid_size1
        ) / (temp_df.bid_size1 + temp_df.ask_size1)

        groups = temp_df.groupby("time_id")
        for g in groups:
            x = expand_to_600(g[1])
            book_train_list.append(x)
        pd.concat(book_train_list, axis=0).to_parquet(os.path.join(BASE_DIRECTORY, 'parquet', f'{id}.parquet'))

In [ ]:
!cp ./optiver-realized-volatility-prediction/parquet/* ./drive/MyDrive/Colab\ Notebooks/Kaggle/preprocessed/

In [ ]:
!cp ./*.csv ./drive/MyDrive/Colab\ Notebooks/Kaggle/preprocessed/